<a href="https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/20231212_nue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Colab Pro V100 (VRAM 16GB)にて動作確認済み

https://huggingface.co/rinna/nue-asr


参考にさせていただいたソース
[@trtd6trtdさん](https://twitter.com/trtd6trtd/status/1734176496011689991?s=12&t=69_MakYffSROXUShicR7xw&fbclid=IwAR1-AGUqfSjeHdkFsIeF-NwUphAhWz61A6wg3mny7vV5u4YM9W8EbDw1yWg)

https://gist.github.com/trtd56/f04625e524d2eba6b6b31f393ce79728



In [1]:
!pip install git+https://github.com/rinnakk/nue-asr.git


  Cloning https://github.com/rinnakk/nue-asr.git to /tmp/pip-req-build-qd2_1owy
  Running command git clone --filter=blob:none --quiet https://github.com/rinnakk/nue-asr.git /tmp/pip-req-build-qd2_1owy
  Resolved https://github.com/rinnakk/nue-asr.git to commit e4207a101711e0920520a0902802767266ec8ea3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.6 MB/s eta 0:00:00
  Created wheel for nue-asr: filename=nue_asr-0.1.0-py3-none-any.whl size=13149 sha256=dc2146d4c6adc598fa6d5669b5c7e0da6142e1f112d594bf8295a58d67096083
  Stored in directory: /tmp/pip-ephem-wheel-cache-0sl2kfn7/wheels/5c/b9/96/b06161b8d32b392f8d9c9fa2615d54b0fafd3d160977e90eac
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-

In [2]:
!pip install pydub
!nvidia-smi

Tue Dec 12 10:15:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!echo "path" > list.tsv
!mkdir clips
# ここで対象となるファイル（ここでは AWS-toned.mp3）をアップロード。
# 最後までファイルがアップロードされたのを確認してから次のステップを実施すること。

In [4]:
import pandas as pd
from pydub import AudioSegment
from tqdm.auto import tqdm
import nue_asr

#pydub のインポート
from pydub import AudioSegment
from pydub.silence import split_on_silence

#mp3ファイルの読み込み(C:/Users/hoge/Musicフォルダに bgm.mp3があることを想定）
sound = AudioSegment.from_file('AWS-toned.mp3', format='mp3')

#mp3の分割（無音部分で区切る場合）
# chunks = split_on_silence(sound, min_silence_len=100, silence_thresh=-55, keep_silence=100)
#分割したデータ毎にファイル出力
#for i, chunk in enumerate(chunks):
#    chunk.export(f'./clips/clips{i}.wav', format='wav')

#時間で区切る場合、ファイル情報の表示
print(f'チャンネル数: {sound.channels}')
print(f'サンプリング周波数: {sound.frame_rate} Hz')
print(f'再生時間（秒）: {sound.duration_seconds} 秒')
print(f'音量: {sound.dBFS} dB')
seg = 20 # 20秒ごとに分割してみる
n_chunks = int(sound.duration_seconds // seg) # seg秒ごとに分割

for i in range(n_chunks):
  start_time = i * seg * 1000  # pydubはミリ秒を使用するため、1000をかける
  end_time = (i+1) * seg * 1000
  chunk = sound[start_time:end_time]
  chunk_path = f"./clips/clips{i}.wav"
  chunk.export(chunk_path, format="wav")
  print(chunk_path)



チャンネル数: 2
サンプリング周波数: 44100 Hz
再生時間（秒）: 2101.315918367347 秒
音量: -17.5912849935689 dB
./clips/clips0.wav
./clips/clips1.wav
./clips/clips2.wav
./clips/clips3.wav
./clips/clips4.wav
./clips/clips5.wav
./clips/clips6.wav
./clips/clips7.wav
./clips/clips8.wav
./clips/clips9.wav
./clips/clips10.wav
./clips/clips11.wav
./clips/clips12.wav
./clips/clips13.wav
./clips/clips14.wav
./clips/clips15.wav
./clips/clips16.wav
./clips/clips17.wav
./clips/clips18.wav
./clips/clips19.wav
./clips/clips20.wav
./clips/clips21.wav
./clips/clips22.wav
./clips/clips23.wav
./clips/clips24.wav
./clips/clips25.wav
./clips/clips26.wav
./clips/clips27.wav
./clips/clips28.wav
./clips/clips29.wav
./clips/clips30.wav
./clips/clips31.wav
./clips/clips32.wav
./clips/clips33.wav
./clips/clips34.wav
./clips/clips35.wav
./clips/clips36.wav
./clips/clips37.wav
./clips/clips38.wav
./clips/clips39.wav
./clips/clips40.wav
./clips/clips41.wav
./clips/clips42.wav
./clips/clips43.wav
./clips/clips44.wav
./clips/clips45.wav
./clip

In [5]:
!ls ./clips/*.wav -v >> list.tsv



# コマンドラインの使い方

以下のコマンドは、コマンドラインインターフェイスを使って音声ファイルを書き起こし(transcribe)ます。音声ファイルは自動的に16kHzにダウンサンプリングされます。



```
!nue-asr audio1.wav
```



In [6]:
# 実験として最初のクリップにコマンドラインアプリから音声書き起こしをしてみる。長すぎるとか、VRAMが足りないとかはあり得る。
!nue-asr ./clips/clips0.wav


config.json: 100% 814/814 [00:00<00:00, 4.61MB/s]
pytorch_model.bin: 100% 7.42G/7.42G [00:30<00:00, 245MB/s]
Some weights of the model checkpoint at rinna/nue-asr were not used when initializing NueASRModel: ['audio_encoder.encoder.pos_conv_embed.conv.weight_v', 'audio_encoder.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing NueASRModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NueASRModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of NueASRModel were not initialized from the model checkpoint at rinna/nue-asr and are newly initialized: ['audio_encoder.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'audio_encoder.encoder.pos_

In [8]:
!pip install deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.4 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.12.4-py3-none-any.whl size=1290644 sha256=8532767afdbb6291ec0c52e2b7cdcdde8ef43fd1c4afa1361d6a83bb06d17203
  Stored in directory: /root/.cache/pip/wheels/7b/0c/52/f464610477b069120f740202a9d84a27f9d7235cbf035c4b75
Successfully built deepspeed


In [9]:
model = nue_asr.load_model("rinna/nue-asr", use_deepspeed=True)
tokenizer = nue_asr.load_tokenizer("rinna/nue-asr")


Some weights of the model checkpoint at rinna/nue-asr were not used when initializing NueASRModel: ['audio_encoder.encoder.pos_conv_embed.conv.weight_g', 'audio_encoder.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing NueASRModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NueASRModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of NueASRModel were not initialized from the model checkpoint at rinna/nue-asr and are newly initialized: ['audio_encoder.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'audio_encoder.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream tas

[2023-12-12 10:25:06,493] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-12-12 10:25:07,762] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.12.4, git-hash=unknown, git-branch=unknown
[2023-12-12 10:25:07,764] [INFO] [logging.py:96:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


Using /root/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py310_cu118/transformer_inference...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu118/transformer_inference/build.ninja...
Building extension module transformer_inference...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


Time to load transformer_inference op: 47.09924554824829 seconds
[2023-12-12 10:25:55,478] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed-Inference config: {'layer_id': 0, 'hidden_size': 2816, 'intermediate_size': 11264, 'heads': 22, 'num_hidden_layers': -1, 'dtype': torch.float16, 'pre_layer_norm': True, 'norm_type': <NormType.LayerNorm: 1>, 'local_rank': -1, 'stochastic_mode': False, 'epsilon': 1e-05, 'mp_size': 1, 'scale_attention': True, 'triangular_masking': True, 'local_attention': False, 'window_size': 1, 'rotary_dim': 128, 'rotate_half': False, 'rotate_every_two': True, 'return_tuple': True, 'mlp_after_attn': False, 'mlp_act_func_type': <ActivationFuncType.GELU: 1>, 'specialized_mode': False, 'training_mp_size': 1, 'bigscience_bloom': False, 'max_out_tokens': 1024, 'min_out_tokens': 1, 'scale_attn_by_inverse_layer_idx': False, 'enable_qkv_quantization': False, 'use_mup': False, 'return_single_tuple': False, 'set_empty_params': False, 'transposed_mode': False, 'use_triton':

Loading extension module transformer_inference...


In [10]:
test_df = pd.read_csv(f"list.tsv", sep="\t")
test_df.head()

TMP_DOWNSAMPLE_AUDIO = "tmp_downsampled_audio.wav"
transcripts = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    print(row.path)
    audio = AudioSegment.from_wav(f"{row.path}")
    downsampled_audio = audio.set_frame_rate(8000)  # 電話音声（8000Hz)の想定
    downsampled_audio.export(TMP_DOWNSAMPLE_AUDIO, format="wav")
    result = nue_asr.transcribe(model, tokenizer, TMP_DOWNSAMPLE_AUDIO)
    transcripts.append(result.text)
    print(result.text)

pd.DataFrame(transcripts, columns=["transcript"]).to_csv("nue_result.csv", index=None)

  0%|          | 0/105 [00:00<?, ?it/s]

./clips/clips0.wav


で観測的にも時間が早いので入りたいと思っているんですがまず最初にですね大阪に鉄棒の部分もあるんですけども衛星データで今というところでマーケットですねどうなっているかというところをですね。
./clips/clips1.wav


こちらですねある雑誌とですね出版のことなんですけども平均でですね35%平均で0円で提供できるというのもああでして恐らくこのITのサービスというかサービスというかですね。
./clips/clips2.wav


MER了解拍手の中で一番今後のビルであろうというこの製鉄所の中でじゃあ帝都大企業がどう見ているかとかキャンパスがどう出ていくかみたいなものをお話しできればと思っていますけどもそこにですね。
./clips/clips3.wav


...

でこちらですねより深い自由な世界に向けて独自レベル開発にも取り組むというところでまた後ほどですねAIが来た第2弾でどのような議論をするのかというところをご紹介させていただくんですけれどもその方です。
./clips/clips100.wav


なので独自で投げられているんですけれども右側で他人が投げられているんですけれども右側で他人が違いや多様性をどのように受け尊重しますか。
./clips/clips101.wav


なので若干こう人とのやりとりってところを想定した場合は違和感があるのでよりこう人と人とのやりとりってところを想定した場合は違和感がある。
./clips/clips102.wav


で第2弾こちらですねマレーシアの方は特別支度をしたんですけれども現在こちらですね逆境をAIというものを第2弾の想定をしております。
./clips/clips103.wav


こちらはですねひざの支柱単独の手術でしたりとかあとは高輪の音声ですね10時間ほどディープなものでいったんですねチューニングなしで音声の作成しておりますのでこれからご紹介させていただきます。
./clips/clips104.wav
ちょっと忘れてたんですけど。
